In [1]:
import numpy as np
import pandas as pd
from Preprocessing import *
import torch.nn as nn
import torch

In [2]:
def data_structure(raw):
    sampling_freq = raw.info['sfreq']
    start_end_secs = np.array([0,4850]) # Fro this dataset data start at 0.000 and end at 4840.166 secs
    start_sample, stop_sample = (start_end_secs * sampling_freq).astype(int)
    df = raw.to_data_frame(picks=['all'], start=start_sample, stop=stop_sample)
    # then save using df.to_csv(...), df.to_hdf(...), etc

    df = df.drop(['time', "STI 014"], axis=1)
    return df

In [3]:
def create_rolling_window(data, window_size=5):
    """
    This function creates a rolling window of a given size from a data frame.
    It assigns the majority label to the window.

    Creator : Niall

    Args:
        data: Dataframe to be windowed
        window_size: The size of the window to be created

    Returns:
        windows: A list of dataframes containing the windows
        labels: A list of labels corresponding to the windows

    """

    windows = []
    labels = []
    data = data.to_numpy()
    for i in range(len(data) - window_size + 1):
        window_data = data[i:i + window_size]
        label = window_data[:, -1][0]  # Assign majority label
        labels.append(label)
        window_data = window_data[:, :-1]
        windows.append(window_data)

    return windows, labels

In [10]:
from torch.utils.data import Dataset

class CustomDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        sample = self.data[index]
        return sample

In [4]:
raw = import_data(r"C:\Users\niall\Desktop\PhD\OpenMIIR-RawEEG_v1\P04-raw.fif")

Opening raw data file C:\Users\niall\Desktop\PhD\OpenMIIR-RawEEG_v1\P04-raw.fif...
Isotrak not found
    Read a total of 1 projection items:
        Average EEG reference (1 x 64)  idle
    Range : 0 ... 2480032 =      0.000 ...  4843.812 secs
Ready.


In [5]:
df = data_structure(raw)

In [6]:
windows, labels = create_rolling_window(df, window_size=5)

In [7]:
data_list = windows[:200]  # Replace with your actual data

# Convert the list of arrays to PyTorch tensors
data_tensors = [torch.Tensor(array) for array in data_list]

# Stack the tensors to create a 3D tensor (200x5x67)
data_tensor = torch.stack(data_tensors, dim=0)

In [11]:
from torch.utils.data import DataLoader

batch_size = 32  # Set your desired batch size
shuffle = True   # You can set this to True if you want to shuffle the data

dataset = CustomDataset(data_tensor)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=shuffle)


In [13]:
for batch in dataloader:
    print(batch.shape)
    # Do stuff with the batch

torch.Size([32, 5, 67])
torch.Size([32, 5, 67])
torch.Size([32, 5, 67])
torch.Size([32, 5, 67])
torch.Size([32, 5, 67])
torch.Size([32, 5, 67])
torch.Size([8, 5, 67])


In [15]:
def make_discriminator_network(
        input_size,
        num_hidden_layers=1,
        num_hidden_units=100,
        num_output_units=1
):
    model = nn.Sequential()
    for i in range(num_hidden_layers):
        model.add_module(
            f'fc_d{i}',
            nn.Linear(input_size, num_hidden_units, bias=False)
        )
        model.add_module(f'relu_d{i}', nn.LeakyReLU())
        model.add_module('dropout', nn.Dropout(p=0.5))
        input_size = num_hidden_units
    model.add_module(f'fc_d{num_hidden_layers}',
                     nn.Linear(input_size, num_output_units))
    model.add_module('sigmoid', nn.Sigmoid())
    return model

In [32]:
disc_model = make_discriminator_network(
    input_size=335,
    num_hidden_layers=1,
    num_hidden_units=100
)

In [33]:
print(disc_model)

Sequential(
  (fc_d0): Linear(in_features=335, out_features=100, bias=False)
  (relu_d0): LeakyReLU(negative_slope=0.01)
  (dropout): Dropout(p=0.5, inplace=False)
  (fc_d1): Linear(in_features=100, out_features=1, bias=True)
  (sigmoid): Sigmoid()
)


In [34]:
input_real = next(iter(dataloader))
input_real = input_real.view(batch_size, -1)

In [35]:
d_proba_real = disc_model(input_real)

In [37]:
d_proba_real.shape

torch.Size([32, 1])